In [1]:
import pandas as pd

In [2]:
file_path='yelpdata.csv'
yelp_df=pd.read_csv(file_path)
yelp_df.head(10)

,Unnamed: 0,latitude,longitude,category,results_total,avg_rating,avg_review_count
0,0,38.046407,-84.497039,beaches,1,1.500000,0.666667
1,1,38.046407,-84.497039,hanggliding,0,0.000000,0.000000
2,2,38.046407,-84.497039,horsebackriding,8,3.000000,0.666667
3,3,38.046407,-84.497039,hiking,4,6.166667,18.666667
4,4,38.046407,-84.497039,hot_air_baloons,20,28.333333,1723.000000
5,5,38.046407,-84.497039,paddleboarding,2,3.166667,10.000000
6,6,38.046407,-84.497039,parasailing,0,0.000000,0.000000
7,7,38.046407,-84.497039,sailing,0,0.000000,0.000000
8,8,38.046407,-84.497039,snorkeling,0,0.000000,0.000000
9,9,38.046407,-84.497039,ziplining,20,28.333333,1723.000000


In [3]:
yelp_df = yelp_df.drop(yelp_df.columns[0], axis=1)
yelp_df.head()

,latitude,longitude,category,results_total,avg_rating,avg_review_count
0,38.046407,-84.497039,beaches,1,1.500000,0.666667
1,38.046407,-84.497039,hanggliding,0,0.000000,0.000000
2,38.046407,-84.497039,horsebackriding,8,3.000000,0.666667
3,38.046407,-84.497039,hiking,4,6.166667,18.666667
4,38.046407,-84.497039,hot_air_baloons,20,28.333333,1723.000000


In [4]:
yelp_df['top_rated'] = yelp_df['avg_rating'] * yelp_df['avg_review_count'] 

# Groupby the geocoordinates and keep only the rows with the highest 5 'top_rated' values
yelp_df= yelp_df.groupby(['latitude', 'longitude']).apply(lambda x: x.nlargest(5, 'top_rated')).reset_index(drop=True)

yelp_df.head(5)

,latitude,longitude,category,results_total,avg_rating,avg_review_count,top_rated
0,-45.032192,168.661,hot_air_baloons,20,29.333333,504.666667,14803.555556
1,-45.032192,168.661,ziplining,20,29.333333,504.666667,14803.555556
2,-45.032192,168.661,transportation,20,29.333333,504.666667,14803.555556
3,-45.032192,168.661,restaurants,20,29.166667,507.000000,14787.500000
4,-45.032192,168.661,nightlife,20,28.333333,194.666667,5515.555556


In [6]:
# Group by latitude and longitude, and aggregate other columns
grouped_df = yelp_df.groupby(['latitude', 'longitude']).agg({'category': lambda x: list(x),
                                                           'results_total': 'sum',
                                                           'avg_rating': 'mean',
                                                           'avg_review_count': 'sum',
                                                           'top_rated': lambda x: list(x)})

# Reset the index to convert group labels back to columns
grouped_df = grouped_df.reset_index()

grouped_df.head()

,latitude,longitude,category,results_total,avg_rating,avg_review_count,top_rated
0,-45.032192,168.661000,"[hot_air_baloons, ziplining, transportation, r...",100,29.100000,2215.666667,"[14803.555555555555, 14803.555555555555, 14803..."
1,-37.814218,144.963161,"[hot_air_baloons, ziplining, transportation, r...",100,28.566667,3400.666667,"[21433.11111111111, 21433.11111111111, 21433.1..."
2,-36.852095,174.763180,"[hot_air_baloons, ziplining, transportation, r...",100,28.966667,2168.666667,"[13747.222222222223, 13747.222222222223, 13747..."
3,-36.718805,142.220951,"[hot_air_baloons, ziplining, transportation, r...",100,24.166667,62.333333,"[427.5, 427.5, 427.5, 269.27777777777777, 67.5]"
4,-35.141813,150.391646,"[restaurants, nightlife, hotels, resorts, beac...",81,5.800000,19.000000,"[329.38888888888886, 5.111111111111111, 4.0, 0..."


In [7]:
# Create a function to get the nth ranked category
def get_nth_category(categories, ranks, n):
    if n < len(categories):
        return categories[n] if ranks[n] else ''
    else:
        return ''

# Get the maximum number of categories
max_categories = max(grouped_df['category'].apply(len))

# Loop through the categories and create separate columns
for i in range(max_categories):
    grouped_df['category{}'.format(i + 1)] = grouped_df.apply(lambda row: get_nth_category(row['category'], row['top_rated'], i), axis=1)

grouped_df.head()

,latitude,longitude,category,results_total,avg_rating,avg_review_count,top_rated,category1,category2,category3,category4,category5
0,-45.032192,168.661000,"[hot_air_baloons, ziplining, transportation, r...",100,29.100000,2215.666667,"[14803.555555555555, 14803.555555555555, 14803...",hot_air_baloons,ziplining,transportation,restaurants,nightlife
1,-37.814218,144.963161,"[hot_air_baloons, ziplining, transportation, r...",100,28.566667,3400.666667,"[21433.11111111111, 21433.11111111111, 21433.1...",hot_air_baloons,ziplining,transportation,restaurants,nightlife
2,-36.852095,174.763180,"[hot_air_baloons, ziplining, transportation, r...",100,28.966667,2168.666667,"[13747.222222222223, 13747.222222222223, 13747...",hot_air_baloons,ziplining,transportation,restaurants,nightlife
3,-36.718805,142.220951,"[hot_air_baloons, ziplining, transportation, r...",100,24.166667,62.333333,"[427.5, 427.5, 427.5, 269.27777777777777, 67.5]",hot_air_baloons,ziplining,transportation,restaurants,hotels
4,-35.141813,150.391646,"[restaurants, nightlife, hotels, resorts, beac...",81,5.800000,19.000000,"[329.38888888888886, 5.111111111111111, 4.0, 0...",restaurants,nightlife,hotels,resorts,
